In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import glob
import pickle

## 데이터프레임 형태 정리

In [3]:
# 경로 설정
path = "/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/data preprocessing/rawdata_for_A3NCF/"

In [4]:
# reviews_three 불러오기 (review 3개 이상 & rating 3개 이상)
df = pd.read_csv(path + "reviews_three.csv")
df.head()

,name,address,user_id,rating,review,cafe
0,공차 이대익스프레스점,대현동,100ck0j,1.0,밑에 후기에 노란색탈색머리 알바 불친절하다고 그랬는데\n진짜 그러네요 내돈주고 사먹...,공차 이대익스프레스점 대현동
1,투썸플레이스 숙대입구역점,남영동,100ck0j,1.0,최애 케이크가 투썸 스트로베리초콜릿케이크라서 엄청 많이 먹어봤는데 이렇게 맛없는 케...,투썸플레이스 숙대입구역점 남영동
2,폴바셋 방배역점,방배동,100ck0j,5.0,직원분들 전부 친절하시고 주차도 가능해서 이용하기에 좋아요:) 주차 관리인 아저씨도...,폴바셋 방배역점 방배동
3,고로커피로스터스,봉천동,100i22n,4.0,커피랑 원두 맛남,고로커피로스터스 봉천동
4,나따오비까 대치점,대치동,100i22n,3.0,에그타르트 클래식 ☆☆☆\n에그타르트 크림치즈 ☆☆☆.5\n\n시나몬향이 나고 빵 ...,나따오비까 대치점 대치동


In [5]:
# 전체 리뷰 개수 (rows 12608, user 2332명, cafe 3884개)
rows = df.shape[0]
user = df["user_id"].unique().size
cafe = df["cafe"].unique().size
print(f"rows: {rows}, user: {user}, cafe: {cafe}")

rows: 12608, user: 2332, cafe: 3884


- 사용할 데이터

|df|df_input|
| --- | --- |
| name | x |
| address | x |
| user_id | userindex |
| rating | rating |
| review | reviews |
| cafe | itemindex |

In [6]:
# name, address 제거
df.drop(columns=["name", "address"], inplace=True)
df.head()

,user_id,rating,review,cafe
0,100ck0j,1.0,밑에 후기에 노란색탈색머리 알바 불친절하다고 그랬는데\n진짜 그러네요 내돈주고 사먹...,공차 이대익스프레스점 대현동
1,100ck0j,1.0,최애 케이크가 투썸 스트로베리초콜릿케이크라서 엄청 많이 먹어봤는데 이렇게 맛없는 케...,투썸플레이스 숙대입구역점 남영동
2,100ck0j,5.0,직원분들 전부 친절하시고 주차도 가능해서 이용하기에 좋아요:) 주차 관리인 아저씨도...,폴바셋 방배역점 방배동
3,100i22n,4.0,커피랑 원두 맛남,고로커피로스터스 봉천동
4,100i22n,3.0,에그타르트 클래식 ☆☆☆\n에그타르트 크림치즈 ☆☆☆.5\n\n시나몬향이 나고 빵 ...,나따오비까 대치점 대치동


In [7]:
# 열 이름 및 순서 변경
df.columns = ['userIndex', 'rating', 'reviews', 'itemIndex']
df = df[['userIndex', 'itemIndex', 'rating', 'reviews']]
df.head()

,userIndex,itemIndex,rating,reviews
0,100ck0j,공차 이대익스프레스점 대현동,1.0,밑에 후기에 노란색탈색머리 알바 불친절하다고 그랬는데\n진짜 그러네요 내돈주고 사먹...
1,100ck0j,투썸플레이스 숙대입구역점 남영동,1.0,최애 케이크가 투썸 스트로베리초콜릿케이크라서 엄청 많이 먹어봤는데 이렇게 맛없는 케...
2,100ck0j,폴바셋 방배역점 방배동,5.0,직원분들 전부 친절하시고 주차도 가능해서 이용하기에 좋아요:) 주차 관리인 아저씨도...
3,100i22n,고로커피로스터스 봉천동,4.0,커피랑 원두 맛남
4,100i22n,나따오비까 대치점 대치동,3.0,에그타르트 클래식 ☆☆☆\n에그타르트 크림치즈 ☆☆☆.5\n\n시나몬향이 나고 빵 ...


## reviews column 전처리
- 문장부호
- stopword 
- 전체 데이터셋에서 10회 이하 등장한 단어들  
=> 제외

### 사전 준비
- review 컬럼 분리
- NLP 관련 library 호출

In [8]:
# df_review: df에서 reviews column만 분리
df_review = df.reviews
df_review[:5]

0    밑에 후기에 노란색탈색머리 알바 불친절하다고 그랬는데\n진짜 그러네요 내돈주고 사먹...
1    최애 케이크가 투썸 스트로베리초콜릿케이크라서 엄청 많이 먹어봤는데 이렇게 맛없는 케...
2    직원분들 전부 친절하시고 주차도 가능해서 이용하기에 좋아요:) 주차 관리인 아저씨도...
3                                            커피랑 원두 맛남
4    에그타르트 클래식 ☆☆☆\n에그타르트 크림치즈 ☆☆☆.5\n\n시나몬향이 나고 빵 ...
Name: reviews, dtype: object

In [9]:
# list_review: df_review의 자료구조 변경 (series -> list)
list_review = df_review.to_list()
list_review[:5]

['밑에 후기에 노란색탈색머리 알바 불친절하다고 그랬는데\n진짜 그러네요 내돈주고 사먹는데 기분개나쁨 다시는 안갈듯',
 '최애 케이크가 투썸 스트로베리초콜릿케이크라서 엄청 많이 먹어봤는데 이렇게 맛없는 케이크는 처음이네요;;; 당황스러울정도임...\n딸기는 엄청 신선한데 케이크 자체는 오래놔뒀던걸 쓴건지 시트는 엄청 푸석푸석해서 포크질 할때마다 부스러기 엄청 나오고 \n크림안에 들어있는 크런치볼은 입에 넣자마자 녹아내림.. 크런치볼은 무슨 소프트볼인줄ㅡㅡ\n더보기',
 '직원분들 전부 친절하시고 주차도 가능해서 이용하기에 좋아요:) 주차 관리인 아저씨도 친절하세요!',
 '커피랑 원두 맛남',
 '에그타르트 클래식 ☆☆☆\n에그타르트 크림치즈 ☆☆☆.5\n\n시나몬향이 나고 빵 부분은 바삭바삭하나 계란 맛이 좀 적게 느껴진다']

In [10]:
# list_review 내 요소의 자료형을 전부 string으로 바꾸기
for i, review in enumerate(list_review):
    review = str(review)
    list_review[i] = review

list_review[:5]

['밑에 후기에 노란색탈색머리 알바 불친절하다고 그랬는데\n진짜 그러네요 내돈주고 사먹는데 기분개나쁨 다시는 안갈듯',
 '최애 케이크가 투썸 스트로베리초콜릿케이크라서 엄청 많이 먹어봤는데 이렇게 맛없는 케이크는 처음이네요;;; 당황스러울정도임...\n딸기는 엄청 신선한데 케이크 자체는 오래놔뒀던걸 쓴건지 시트는 엄청 푸석푸석해서 포크질 할때마다 부스러기 엄청 나오고 \n크림안에 들어있는 크런치볼은 입에 넣자마자 녹아내림.. 크런치볼은 무슨 소프트볼인줄ㅡㅡ\n더보기',
 '직원분들 전부 친절하시고 주차도 가능해서 이용하기에 좋아요:) 주차 관리인 아저씨도 친절하세요!',
 '커피랑 원두 맛남',
 '에그타르트 클래식 ☆☆☆\n에그타르트 크림치즈 ☆☆☆.5\n\n시나몬향이 나고 빵 부분은 바삭바삭하나 계란 맛이 좀 적게 느껴진다']

In [11]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [913 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [12]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [13]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm
from konlpy.tag import Okt

In [14]:
okt = Okt()

### 품사를 통한 stopword 제거 
- 문장부호, 조사, 접미사, 선어말어미, 어미, 외국어 및 기타 기호, 이메일, URL
- 참고: [형태소 분석기 품사 태그 표](https://velog.io/@hsoh0423/%ED%98%95%ED%83%9C%EC%86%8C-%EB%B6%84%EC%84%9D%EA%B8%B0-%ED%92%88%EC%82%AC-%ED%83%9C%EA%B7%B8%ED%91%9C)

In [15]:
def stopwordTokenization(sentence):
    stop = ['Punctuation', 'Josa', 'Suffix', 'PreEomi', 'Eomi', 'Foreign', 'Email', 'URL']
    token = okt.pos(sentence)
    token = [i for i, j in token if j not in stop]
    return token

In [16]:
review_tokenized = []

for i in tqdm(list_review):
    review_tokenized.append(stopwordTokenization(i))

  0%|          | 0/12608 [00:00<?, ?it/s]

In [17]:
review_tokenized[:5]

[['밑',
  '후기',
  '노란색',
  '탈색',
  '머리',
  '알바',
  '불친절하다고',
  '그랬는데',
  '진짜',
  '그러네요',
  '내',
  '돈',
  '주고',
  '사먹는데',
  '기분',
  '나쁨',
  '다시는',
  '안',
  '갈듯'],
 ['최애',
  '케이크',
  '투썸',
  '스트로베리',
  '초콜릿케이크',
  '엄청',
  '많이',
  '먹어',
  '봤는데',
  '이렇게',
  '맛없는',
  '케이크',
  '처음',
  '요',
  '당황',
  '스러울',
  '정도',
  '임',
  '딸기',
  '엄청',
  '신선한데',
  '케이크',
  '자체',
  '오래',
  '놔뒀던걸',
  '쓴',
  '건지',
  '시트',
  '엄청',
  '푸석푸석해서',
  '포크',
  '질',
  '할',
  '때',
  '부스러기',
  '엄청',
  '나오고',
  '크림',
  '안',
  '들어있는',
  '크런치',
  '볼',
  '입',
  '넣자마자',
  '녹아내림',
  '크런치',
  '볼',
  '무슨',
  '소프트볼',
  '줄',
  'ㅡㅡ',
  '더',
  '보기'],
 ['직원',
  '전부',
  '친절하시고',
  '주차',
  '가능해서',
  '이용',
  '하기에',
  '좋아요',
  '주차',
  '관리인',
  '아저씨',
  '친절하세요'],
 ['커피', '원두', '맛남'],
 ['에그타르트',
  '클래식',
  '에그타르트',
  '크림',
  '치즈',
  '5',
  '시나몬',
  '향',
  '나고',
  '빵',
  '부분',
  '바삭바삭하나',
  '계란',
  '맛',
  '좀',
  '적게',
  '느껴진다']]

### 전체 데이터셋에서 3회 이하 등장한 단어들 제거

In [18]:
# bag-of-word
def build_bag_of_words(review):

    word_count = {}
    bow = []
    
    for sentence in review:     # review_tokenized에서 list 하나씩 꺼내기
        for word in sentence:   # 꺼낸 list에서 word 하나씩 꺼내기
            if word not in word_count.keys():
                word_count[word] = len(word_count)  
                # BoW에 전부 기본값 1을 넣는다.
                bow.insert(len(word_count) - 1, 1)
            else:
                # 재등장하는 단어의 인덱스
                index = word_count.get(word)
                # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더한다.
                bow[index] = bow[index] + 1

    return word_count, bow

In [19]:
vocab, bow = build_bag_of_words(review_tokenized)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'밑': 0, '후기': 1, '노란색': 2, '탈색': 3, '머리': 4, '알바': 5, '불친절하다고': 6, '그랬는데': 7, '진짜': 8, '그러네요': 9, '내': 10, '돈': 11, '주고': 12, '사먹는데': 13, '기분': 14, '나쁨': 15, '다시는': 16, '안': 17, '갈듯': 18, '최애': 19, '케이크': 20, '투썸': 21, '스트로베리': 22, '초콜릿케이크': 23, '엄청': 24, '많이': 25, '먹어': 26, '봤는데': 27, '이렇게': 28, '맛없는': 29, '처음': 30, '요': 31, '당황': 32, '스러울': 33, '정도': 34, '임': 35, '딸기': 36, '신선한데': 37, '자체': 38, '오래': 39, '놔뒀던걸': 40, '쓴': 41, '건지': 42, '시트': 43, '푸석푸석해서': 44, '포크': 45, '질': 46, '할': 47, '때': 48, '부스러기': 49, '나오고': 50, '크림': 51, '들어있는': 52, '크런치': 53, '볼': 54, '입': 55, '넣자마자': 56, '녹아내림': 57, '무슨': 58, '소프트볼': 59, '줄': 60, 'ㅡㅡ': 61, '더': 62, '보기': 63, '직원': 64, '전부': 65, '친절하시고': 66, '주차': 67, '가능해서': 68, '이용': 69, '하기에': 70, '좋아요': 71, '관리인': 72, '아저씨': 73, '친절하세요': 74, '커피': 75, '원두': 76, '맛남': 77, '에그타르트': 78, '클래식': 79, '치즈': 80, '5': 81, '시나몬': 82, '향': 83, '나고': 84, '빵': 85, '부분': 86, '바삭바삭하나': 87, '계란': 88, '맛': 89, '좀': 90, '적게': 91, '느껴진다': 92, '나': 93, '는': 94, 

In [20]:
# vocab: dict -> dataframe
df_vocab = pd.DataFrame(vocab, index=["index"])
df_vocab = df_vocab.transpose()
df_vocab["word"] = df_vocab.index
df_vocab.set_index(keys="index", inplace=True)
df_vocab

,word
index,
0,밑
1,후기
2,노란색
3,탈색
4,머리
...,...
22711,인건비
22712,나가겠지만
22713,단팥빵


In [21]:
# bow: list -> series
df_bow = pd.Series(bow)
df_bow

0        56
1        78
2         1
3         1
4        46
         ..
22711     1
22712     1
22713     5
22714     1
22715     1
Length: 22716, dtype: int64

In [22]:
# df_vocab + df_bow
df_vocab["count"] = df_bow
df_vocab.sort_values(["count"], ascending=False, inplace=True)
df_vocab

,word,count
index,,
75,커피,2774
89,맛,2514
96,너무,1972
62,더,1842
149,카페,1463
...,...,...
13906,잔고,1
13907,참았음,1
13910,나가신다,1


In [23]:
higher = df_vocab[df_vocab["count"] >= 3]
lower = df_vocab[df_vocab["count"] < 3]
higher_sum = higher["count"].sum()
lower_sum = lower["count"].sum()
print("3번 이상 출현한 단어들의 총 개수:", higher_sum, "\n" "3번 미만 출현한 단어들의 총 개수:", lower_sum)

3번 이상 출현한 단어들의 총 개수: 165359 
3번 미만 출현한 단어들의 총 개수: 19218


In [24]:
# df_vocab: 전체 리뷰에서 3번 이상 등장한 단어들의 bow
df_vocab = df_vocab[df_vocab["count"] >= 3]
df_vocab

,word,count
index,,
75,커피,2774
89,맛,2514
96,너무,1972
62,더,1842
149,카페,1463
...,...,...
5622,만드는듯,3
9981,남긴,3
3787,느끼고,3


In [25]:
# list_vocab: 전체 리뷰의 단어 집합
list_vocab = list(df_vocab["word"])
len(list_vocab)

6849

In [26]:
# review_tokenized에서 list_vocab에 없는 단어들(3회 미만 등장 단어들) 삭제
for review in review_tokenized:             # 전체 리뷰에서 하나의 리뷰를 꺼낸다
    for i, word in enumerate(review):       # 하나의 리뷰에서 하나의 토큰(word)을 꺼낸다
        if word not in list_vocab:          # 그 토큰이 list_vocab에 있는 단어가 아니라면
            #print(word)             
            del review[i]                   # 해당 토큰 삭제

In [27]:
review_tokenized[:5]

[['밑',
  '후기',
  '탈색',
  '머리',
  '알바',
  '불친절하다고',
  '그랬는데',
  '진짜',
  '그러네요',
  '내',
  '돈',
  '주고',
  '사먹는데',
  '기분',
  '나쁨',
  '다시는',
  '안',
  '갈듯'],
 ['최애',
  '케이크',
  '투썸',
  '스트로베리',
  '엄청',
  '많이',
  '먹어',
  '봤는데',
  '이렇게',
  '맛없는',
  '케이크',
  '처음',
  '요',
  '당황',
  '정도',
  '임',
  '딸기',
  '엄청',
  '케이크',
  '자체',
  '오래',
  '쓴',
  '건지',
  '시트',
  '엄청',
  '포크',
  '질',
  '할',
  '때',
  '부스러기',
  '엄청',
  '나오고',
  '크림',
  '안',
  '들어있는',
  '크런치',
  '볼',
  '입',
  '녹아내림',
  '크런치',
  '볼',
  '무슨',
  '줄',
  'ㅡㅡ',
  '더',
  '보기'],
 ['직원',
  '전부',
  '친절하시고',
  '주차',
  '가능해서',
  '이용',
  '하기에',
  '좋아요',
  '주차',
  '관리인',
  '아저씨',
  '친절하세요'],
 ['커피', '원두', '맛남'],
 ['에그타르트',
  '클래식',
  '에그타르트',
  '크림',
  '치즈',
  '5',
  '시나몬',
  '향',
  '나고',
  '빵',
  '부분',
  '계란',
  '맛',
  '좀',
  '적게',
  '느껴진다']]

In [28]:
# 잘 삭제되었는지 확인
for review in review_tokenized:     # 전체 리뷰에서 하나의 리뷰를 꺼낸다
    for word in review:             # 하나의 리뷰에서 하나의 토큰(word)을 꺼낸다
        if word not in list_vocab:  # 그 토큰이 list_vocab에 있는 단어가 아니라면
            print(word)             # 출력해줘 -> 아무것도 출력 안되면 OK

탈색
녹아내림
ㅜㅋㅋㅋ
4월
이인
엽서
악합니다
구독
미각
싫다면
남는샷
민망함을
해주던지
떨어진다
앉아있더라
몰겠네요
도구
재미있을
새것의
상시
벤치
바라볼
않다는
듁스
부리는
ㅠㅠㅠㅠㅠㅠㅠㅠㅠ
씻어내고
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
시간여행
ㅠㅜㅜ
찾으
#문래동카페
신림역
신림로
쏭
강남대로
유당불내증
캐슈넛
넣었습니다
관악구
신다면
사평대로
212
사오는데
맛있기보다는
쌀쿠아즈
숙성
제거
느꼈네요
만든것
단길
불쾌함
신나하다가
느껴졌다
독
즐거웠다
어지간하면
규용
갈리지도
될텐데
자고
되는지는
뽀
부피
만드는것보다
야박한
상치
않겠다는
웃긴건
땅땅
로빈
우지
탑승
#로스터리카페
291-26
가능한데
잡아
바꿀
더것
콘서트
있는데로
우기시더니
똑같을거라면서
멀쩡해지는
쩔어요
피클
진하구
틀어주면
모아에서
카토
재밌네
같긴한데
원점
싶긴
제비
향초
항의
소음기
따지고있고
러프
속지
돌아도
찾아보니
6/14
눌러
crumble
푸드코트
볶음
미소시루
개꿀맛
료워
따리
들어서면
본건데
날땐
없겠다
산화
박힌
만들었던
재질이었다
매는
꽉차서
퓨
현상
않더라고요
콜
시킬거면
없다네요
가는거라면
같더라
물었더니
되죠
이었나
이었나
08
봇
즐기면서
쩡
투하
팔방
아니시라면
살고있어서
대형견
스트로
물어보는
찍어주시는
물어보는
찍어주시는
으그
줄알
프루츠
씹는
보일듯
넼
현지
끼어
어둑한
출입문
그려진
어리둥절했음
살펴
성공한
알았다더니
우비
쩡
파시네요
안되있어서
나왔던거
아낄거
올라
가게였는데
위아래
연하
신선한것
바라지는
비아
수유역
나눠져서
프린팅
여쭙자
받아들이고
석양
있는것만
어요
대량
몰려와서
터눈
했을듯
터미널
진흙
흐물거려
현미
로딩
씨름
해유
모르겟구
림
생겼고
공중목욕탕
않겠음
호스
달라붙지
빌려주셔서
블치케
물어봐
듣는
빠방해
만들어주셨어요
타운
본데요
의심스러울
문화유산
부속
사라지더니
22.3월
묘옹
묘옹무응
ㅠㅠㅠㅠㅠㅠㅠㅠㅠ
고무신
놀다가
힘들다고
얼어있으니
주더
당하고
뎀
들이여요
껜
건데
고욤
끄도
완벽하고
바랐는데
만질
빡치는

### 전처리 완료 후 다시 df에 합치기

In [29]:
# 토큰 단위로 쪼개져있던 리뷰를 문장으로 다시 합쳐주기
review_list = []

for review in review_tokenized:
    sentence = " ".join(review)
    review_list.append(sentence)

review_list[:5]

['밑 후기 탈색 머리 알바 불친절하다고 그랬는데 진짜 그러네요 내 돈 주고 사먹는데 기분 나쁨 다시는 안 갈듯',
 '최애 케이크 투썸 스트로베리 엄청 많이 먹어 봤는데 이렇게 맛없는 케이크 처음 요 당황 정도 임 딸기 엄청 케이크 자체 오래 쓴 건지 시트 엄청 포크 질 할 때 부스러기 엄청 나오고 크림 안 들어있는 크런치 볼 입 녹아내림 크런치 볼 무슨 줄 ㅡㅡ 더 보기',
 '직원 전부 친절하시고 주차 가능해서 이용 하기에 좋아요 주차 관리인 아저씨 친절하세요',
 '커피 원두 맛남',
 '에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적게 느껴진다']

In [30]:
# review_series: review_tokenized의 자료구조 변경 (list -> series)
review_series = pd.Series(review_list)
review_series.head()

0    밑 후기 탈색 머리 알바 불친절하다고 그랬는데 진짜 그러네요 내 돈 주고 사먹는데 ...
1    최애 케이크 투썸 스트로베리 엄청 많이 먹어 봤는데 이렇게 맛없는 케이크 처음 요 ...
2      직원 전부 친절하시고 주차 가능해서 이용 하기에 좋아요 주차 관리인 아저씨 친절하세요
3                                             커피 원두 맛남
4    에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적...
dtype: object

In [31]:
# df의 reviews 컬럼을 review_series로 변경
df["reviews"] = review_series
df.head()

,userIndex,itemIndex,rating,reviews
0,100ck0j,공차 이대익스프레스점 대현동,1.0,밑 후기 탈색 머리 알바 불친절하다고 그랬는데 진짜 그러네요 내 돈 주고 사먹는데 ...
1,100ck0j,투썸플레이스 숙대입구역점 남영동,1.0,최애 케이크 투썸 스트로베리 엄청 많이 먹어 봤는데 이렇게 맛없는 케이크 처음 요 ...
2,100ck0j,폴바셋 방배역점 방배동,5.0,직원 전부 친절하시고 주차 가능해서 이용 하기에 좋아요 주차 관리인 아저씨 친절하세요
3,100i22n,고로커피로스터스 봉천동,4.0,커피 원두 맛남
4,100i22n,나따오비까 대치점 대치동,3.0,에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적...


### 각 컬럼별로 자료형 맞춰주기

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12608 entries, 0 to 12607
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userIndex  12608 non-null  object 
 1   itemIndex  12608 non-null  object 
 2   rating     12608 non-null  float64
 3   reviews    12608 non-null  object 
dtypes: float64(1), object(3)
memory usage: 394.1+ KB


#### rating

In [33]:
# rating: float64 -> int64
df = df.astype({"rating":int})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12608 entries, 0 to 12607
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userIndex  12608 non-null  object
 1   itemIndex  12608 non-null  object
 2   rating     12608 non-null  int64 
 3   reviews    12608 non-null  object
dtypes: int64(1), object(3)
memory usage: 394.1+ KB


In [34]:
pd.Series(df["userIndex"].unique())

0       100ck0j
1       100i22n
2       100q04c
3       101ag5k
4       101iuk0
         ...   
2327     vtgq00
2328     vua8j4
2329     vunrjc
2330     vur3vt
2331     vv73qf
Length: 2332, dtype: object

In [35]:
# df_userIndex: 기존 userIndex를 숫자 인덱스로 매칭
df_userIndex = pd.DataFrame(df["userIndex"].unique())
df_userIndex["NumberIndex"] = df_userIndex.index
df_userIndex.columns = ["StringIndex", "NumberIndex"]
df_userIndex.head()

,StringIndex,NumberIndex
0,100ck0j,0
1,100i22n,1
2,100q04c,2
3,101ag5k,3
4,101iuk0,4


In [36]:
# df_itemIndex: 기존 itemIndex를 숫자 인덱스로 매칭
df_itemIndex = pd.DataFrame(df["itemIndex"].unique())
df_itemIndex["NumberIndex"] = df_itemIndex.index
df_itemIndex.columns = ["StringIndex", "NumberIndex"]
df_itemIndex.head()

,StringIndex,NumberIndex
0,공차 이대익스프레스점 대현동,0
1,투썸플레이스 숙대입구역점 남영동,1
2,폴바셋 방배역점 방배동,2
3,고로커피로스터스 봉천동,3
4,나따오비까 대치점 대치동,4


In [37]:
# df_userIndex, df_itemIndex 저장
df_userIndex.to_csv('/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/data preprocessing/rawdata_for_A3NCF/review_three_user.csv', index=False)
df_itemIndex.to_csv('/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/data preprocessing/rawdata_for_A3NCF/review_three_item.csv', index=False)

In [38]:
# 저장한 csv 파일 확인
_user = pd.read_csv('/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/data preprocessing/rawdata_for_A3NCF/review_three_user.csv')
_item = pd.read_csv('/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/data preprocessing/rawdata_for_A3NCF/review_three_item.csv')
_item[:5]

,StringIndex,NumberIndex
0,공차 이대익스프레스점 대현동,0
1,투썸플레이스 숙대입구역점 남영동,1
2,폴바셋 방배역점 방배동,2
3,고로커피로스터스 봉천동,3
4,나따오비까 대치점 대치동,4


In [39]:
# dict_userIndex, dict_itemIndex: df_userIndex, df_itemIndex의 dictionary 버전
dict_userIndex = df_userIndex.set_index("StringIndex").T.to_dict("records")
dict_userIndex = dict_userIndex[0]
dict_itemIndex = df_itemIndex.set_index("StringIndex").T.to_dict("records")
dict_itemIndex = dict_itemIndex[0]
type(dict_userIndex), type(dict_itemIndex)

(dict, dict)

In [40]:
col_user = list(df["userIndex"])
col_item = list(df["itemIndex"])

In [41]:
for i, user in enumerate(col_user):
    for key in dict_userIndex.keys():
        if user == key:
            col_user[i] = dict_userIndex[key]

In [42]:
for i, item in enumerate(col_item):
    for key in dict_itemIndex.keys():
        if item == key:
            col_item[i] = dict_itemIndex[key]

#### userIndex, itemIndex

In [43]:
# userIndex, itemIndex: object -> int
df["userIndex"] = col_user
df["itemIndex"] = col_item

In [44]:
df.head()

,userIndex,itemIndex,rating,reviews
0,0,0,1,밑 후기 탈색 머리 알바 불친절하다고 그랬는데 진짜 그러네요 내 돈 주고 사먹는데 ...
1,0,1,1,최애 케이크 투썸 스트로베리 엄청 많이 먹어 봤는데 이렇게 맛없는 케이크 처음 요 ...
2,0,2,5,직원 전부 친절하시고 주차 가능해서 이용 하기에 좋아요 주차 관리인 아저씨 친절하세요
3,1,3,4,커피 원두 맛남
4,1,4,3,에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적...


In [45]:
df.columns = [0, 1, 2, 3]
df

,0,1,2,3
0,0,0,1,밑 후기 탈색 머리 알바 불친절하다고 그랬는데 진짜 그러네요 내 돈 주고 사먹는데 ...
1,0,1,1,최애 케이크 투썸 스트로베리 엄청 많이 먹어 봤는데 이렇게 맛없는 케이크 처음 요 ...
2,0,2,5,직원 전부 친절하시고 주차 가능해서 이용 하기에 좋아요 주차 관리인 아저씨 친절하세요
3,1,3,4,커피 원두 맛남
4,1,4,3,에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적...
...,...,...,...,...
12603,2330,266,5,직원 친절하세요 음료 맛있어요
12604,2331,2152,5,아주 좋은 호텔 가장 아쉽다 여기 시설 오래 하는 사람 있는데 이 호텔 2014년 ...
12605,2331,3883,1,2021년 11월 6일 추가 내용 이 빵집 폐업 한 것 생각 등록 된 없어 진 것 ...
12606,2331,3097,1,이 곳 커피 맛 평범한데 가게 엄청 좁고 뒤쪽 편하게 먹을 수 있는 가게 안쪽 젊은...


In [46]:
# 최종 확인
# A3NCF_dataset_test에서 확인한 df_tm.info()와 동일해야함
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12608 entries, 0 to 12607
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       12608 non-null  int64 
 1   1       12608 non-null  int64 
 2   2       12608 non-null  int64 
 3   3       12608 non-null  object
dtypes: int64(3), object(1)
memory usage: 394.1+ KB


## 전처리 완료된 데이터 내보내기

In [47]:
# 모든 전처리가 완료된 df 내보내기 !
#df.to_csv("/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/topic model/data/A3NCF_topicmodel_input_df_3.csv", 
#          index=False)           # row명 (index) 저장하지 않기

In [48]:
# 내보낸 df 확인
#df = pd.read_csv("/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/topic model/data/A3NCF_topicmodel_input_df_3.csv")
#df

In [49]:
# hoxy..?
df.to_csv('/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/topic model/data/A3NCF_topicmodel_input_df_3.dat', 
          index=False)           # row명 (index) 저장하지 않기

In [50]:
# hehe
df = pd.read_csv("/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/topic model/data/A3NCF_topicmodel_input_df_3.dat")
df

,0,1,2,3
0,0,0,1,밑 후기 탈색 머리 알바 불친절하다고 그랬는데 진짜 그러네요 내 돈 주고 사먹는데 ...
1,0,1,1,최애 케이크 투썸 스트로베리 엄청 많이 먹어 봤는데 이렇게 맛없는 케이크 처음 요 ...
2,0,2,5,직원 전부 친절하시고 주차 가능해서 이용 하기에 좋아요 주차 관리인 아저씨 친절하세요
3,1,3,4,커피 원두 맛남
4,1,4,3,에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적...
...,...,...,...,...
12603,2330,266,5,직원 친절하세요 음료 맛있어요
12604,2331,2152,5,아주 좋은 호텔 가장 아쉽다 여기 시설 오래 하는 사람 있는데 이 호텔 2014년 ...
12605,2331,3883,1,2021년 11월 6일 추가 내용 이 빵집 폐업 한 것 생각 등록 된 없어 진 것 ...
12606,2331,3097,1,이 곳 커피 맛 평범한데 가게 엄청 좁고 뒤쪽 편하게 먹을 수 있는 가게 안쪽 젊은...
